## Gridworld问题（Reinforcing learning-Introduction）

In [1]:
import numpy as np

### Step1 定义给定随机分布，策略评估的算法
- **迭代公式：$v_{k+1}(s)=\sum_{a \in A} \pi(a | s)\left(R_{s}^{a}+\gamma \sum_{s^{\prime} \in S} P_{s s^{\prime}}^{a} v_{k}\left(s^{\prime}\right)\right)\quad$(1)**  

In [2]:
class PolicyEvaluation:
    def __init__(self,gamma):
        self.gamma = gamma          #衰减系数
    '''
        更新一个状态的价值函数(私有成员)
        i，j:指定状态位置
        policy:策略
        stateValue:状态价值
    '''
    def __calcaluate_one_state_value(self,i,j,policy,stateValue):
        reward = -1                                            #所有动作的即时奖励
        value = np.zeros((4,1),dtype=np.float64)
        direction = np.array([1,0,-1,0,0,-1,0,1]).reshape(4,2) #定义四个方向：上下左右    
        #得到所有可能的状态价值
        num = 0
        for dir1 in direction:                                #遍历该状态的上下左右四个状态
            tempi = i + dir1[0]
            tempj = j + dir1[1]
            if tempi >= 0 and tempi <= 3 and tempj >= 0 and tempj <= 3:
                value[num] = stateValue[tempi,tempj]
            else:                                             #边界格往外走，直接回到原本的格子（状态）
                value[num] = stateValue[i,j]
            num = num+1
        
        #根据公式(1),计算状态价值
        assert(value.shape == (4,1))
        
        res = np.dot( policy[i,j,:].reshape(1,4), self.gamma*value+ reward )
        return res.squeeze()                                          #返回计算出值      
    '''
    Iterative Policy Evaluation
    利用动态规划的方法在给定策略的情况下求出状态价值：
    policy:策略
    stateValue:初始的状态价值
    iterNum:迭代次数 
    '''
    def policyEvaluation(self,policy,stateValue,iterNum = 1):
        
        init_value = stateValue.copy()
        newStateValue = stateValue.copy()     #新的状态价值
        
        for num in range(iterNum):           #迭代次数
            for i in range(4):
                for j in range(4):
                    if i == 0 and j == 0:
                        continue
                    elif i == 3 and j == 3:
                        continue
                    else:
                        newStateValue[i,j] = self.__calcaluate_one_state_value(i,j,policy,init_value)
            init_value = newStateValue.copy()     
        return newStateValue          #返回经过迭代的价值函数

### Step2 定义参数
- **衰减因子$\gamma = 1$ **
- **所有状态得到的即时奖励都是$R=-1$ **
- **每个格子都是固定的，所以可行的状态转换概率$P_{ss'}^{a}=1$(状态s采取动作a转移到一个状态s'的概率)**

In [3]:
gamma = 1.
policy = np.zeros((4,4,4),dtype = np.float64)  #存放策略
#初始化策略
assert(policy.shape == (4,4,4))               #确保存放策略数组的维度,四个方向的概率值an'zha
policy = policy+0.25                           #利用Python机制让每个格子向上下左anzha右转移的概率为0.25
policy[0,0,:] = 0
policy[3,3,:] = 0
stateValue = np.zeros((4,4),dtype=np.float64)  #用于存储状态价值

### Step3 运行算法计算给定随机策略的最终状态价值

In [4]:
Algorithm = PolicyEvaluation(gamma)
iterNum = 1000
newStateValue = Algorithm.policyEvaluation(policy,stateValue,iterNum)
np.set_printoptions(formatter={'float':'{:.4f}'.format})
print(newStateValue)

[[0.0000 -14.0000 -20.0000 -22.0000]
 [-14.0000 -18.0000 -20.0000 -20.0000]
 [-20.0000 -20.0000 -18.0000 -14.0000]
 [-22.0000 -20.0000 -14.0000 0.0000]]


** 当迭代次数k = 1000,状态价值就已经收敛了，其精确到小数点4位的状态价值如上所示，与PPT给出结果一致 **

## MDP:Optimal Gridword问题
- ** 当采取动作偏离地图范围时，回到原来格子，即时奖励是-1**
- ** 状态A采取任何动作都转移到状态A'，得到的即时奖励是+10**
- ** 状态B采取任何动作都转移到状态B',得到的即是奖励是+5**
- ** 其余的动作，得到的即时奖励是0**
- ** 衰减系数$\gamma = 0.9$**

### Step1:定义求取最优策略的算法

In [85]:
class BestPolicy:
    def __init__(self,gamma):
        self.gamma = gamma          #衰减系数

    def __calculate_one_state_value(self,i,j,policy,stateValue):
        reward = np.zeros((4,1),dtype=np.float64)               #默认每个状态的四个动作即时奖励为0
        value = np.zeros((4,1),dtype=np.float64)            
        direction = np.array([1,0,-1,0,0,-1,0,1]).reshape(4,2) #定义四个方向，顺序为下上左右    
        num = 0
        for dir1 in direction:                                #遍历该状态的下上左右四个状态
            
            tempi = i + dir1[0]                                #i是行，j是列
            tempj = j + dir1[1]
            if tempi >= 0 and tempi <= 4 and tempj >= 0 and tempj <= 4:
                value[num] = stateValue[tempi,tempj]
            else:                                             #边界格往外走，直接回到原本的格子（状态）
                value[num] = stateValue[i,j]
                if tempi < 0 :   #上越界
                    reward[0] = -1
                elif tempi > 4:  #下越界
                    reward[1] = -1
                elif tempj < 0:  #左越界
                    reward[2] = -1
                elif tempj > 4:  #右越界
                    reward[3] = -1      
            num = num+1
        
        #根据公式(1),计算状态价值
        
        if i == 0 and j == 1:           #当前是状态A
            reward = np.full((4,1),10)
            value = np.full((4,1),stateValue[4,2])
        elif i == 0 and j == 3:         #当前是状态B
            reward = np.full((4,1),5)
            value = np.full((4,1),stateValue[2,3])
        
        #print(i,j,reward)
        assert(value.shape == (4,1))
        assert(reward.shape == (4,1))
        
        res = np.dot(policy[i,j,:].reshape(1,4), self.gamma*value+ reward )
        return res.squeeze()                                          #返回计算出值      
    
    def policyEvaluation(self,policy,stateValue,iterNum = 1):
        
        init_value = stateValue.copy()
        newStateValue = stateValue.copy()     #新的状态价值
        
        for num in range(iterNum):           #迭代次数
            for i in range(5):
                for j in range(5):
                    newStateValue[i,j] = self.__calculate_one_state_value(i,j,policy,init_value)
            init_value = newStateValue.copy()   
        return newStateValue          #返回经过迭代的价值函数
    
    #计算一个状态的概略分布
    def __calculatePolicy(self,i,j,stateValue): 
        maxv = 0.
        curr_value = np.zeros((4,1),dtype=np.float64)
        assert(curr_value.shape == (4,1))
        res = np.zeros((4,1),dtype=np.float64)
        assert(res.shape == (4,1))
        
        direction = np.array([1,0,-1,0,0,-1,0,1]).reshape(4,2) #定义四个方向，顺序为下上左右    
        num = 0
        
        for dir1 in direction:                                #遍历该状态的上下左右四个状态
            tempi = i + dir1[0]                                #i是行，j是列
            tempj = j + dir1[1]
            
            if tempi >= 0 and tempi <= 4 and tempj >= 0 and tempj <= 4:
                curr_value[num] = stateValue[tempi,tempj]
            else:
                curr_value[num] = stateValue[i,j]
                
            #边界格往外走，直接回到原本的格子（状态）  
            num = num+1        
        
        maxv = curr_value[0]
        for tmp in curr_value:          #找出最大的状态价值函数
            if tmp > maxv:
                maxv = tmp
                
       
        true_num = 0
        flagList = []         
        for tmp in curr_value:
            if maxv - tmp < 0.1:   #状态价值的数值近似相等
                flagList.append(True)
                true_num = true_num+1
            else:
                flagList.append(False)
        
        #计算四个方向的概略分布
        prob = 1.0/true_num      
        num = 0
        for tmp in flagList:
            if tmp == True:
                res[num] = prob
            else:
                res[num] = 0.0
            num = num + 1
        return res
             
    def policyImproverment(self,stateValue,policy):
        newPolicy = policy.copy()
        for row in range(5):
            for col in range(5):
                if row == 0 and col == 1:       #状态A的策略不需要更新，已是最优
                    continue
                elif row == 0 and col == 3:     #状态B的策略不需要更新，已是最优
                    continue
                else:
                    newPolicy[row,col,:] = self.__calculatePolicy(row,col,stateValue).squeeze().copy()
        return newPolicy
    def print_policy(self,policy):
        allList = []
        dirList = ["下","上","左","右"]
        
        for i in range(5):
            for j in range(5):
                num = 0
                for dir1 in dirList:
                    if policy[i,j,num] > 0:
                        allList.append(dirList[num])
                    else:
                        allList.append("-")
                    num += 1
        #打印方向
        num = 0
        j = 0
        while j < len(allList):
            for i in range(j,j+4):
                print(allList[i],end = "")
            print(" ",end = "")
            j += 4
            num += 1
            if num%5 == 0:
                print("")     

### Step2 定义参数

In [87]:
gamma = 0.9   #衰减系数
policy = np.zeros((5,5,4),dtype = np.float64)  #存放策略
#初始化策略
assert(policy.shape == (5,5,4))               #确保存放策略数组的维度,四个方向的概率值
policy = policy+0.25                           #利用Python机制让每个格子向上下左右转移的概率为0.25
stateValue = np.zeros((5,5),dtype=np.float64)  #用于存储状态价值

### Step3 测试策略评价函数是否正确

In [88]:
Algorithm1 = BestPolicy(gamma)
iterNum = 50
newStateValue = Algorithm1.policyEvaluation(policy,stateValue,iterNum)
#np.set_printoptions(formatter={'float':'{:.4f}'.format})
print(newStateValue)

[[3.3681 8.8986 4.4684 5.3293 1.4976]
 [1.5565 3.0344 2.2741 1.9177 0.5535]
 [0.0695 0.7571 0.6863 0.3659 -0.3978]
 [-0.9632 -0.4257 -0.3472 -0.5802 -1.1789]
 [-1.8507 -1.3387 -1.2239 -1.4187 -1.9716]]


### Step4 测试策略提升函数是否正确

In [89]:
print("origin policy:")
print(policy)
print("---------------------------------")
newPolicy = Algorithm1.policyImproverment(newStateValue,policy)
print("new policy:")
print(newPolicy)
#print(newPolicy)
#Algorithm1.print_policy(newPolicy)

origin policy:
[[[0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]]

 [[0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]]

 [[0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]]

 [[0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]]

 [[0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]
  [0.2500 0.2500 0.2500 0.2500]]]
---------------------------------
new policy:
[[[0.0000 0.0000 0.0000 1.0000]
  [0.2500 0.2500 0.2500 0.2500]
  [0.0000 0.0000 1.0000 0.0000]
  [0.2500 0.2500 0.2500 0.2500]
 

### Step5 结合策略评估算法与策略提升算法寻找最优策略

In [96]:
Test = BestPolicy(gamma)
iterNum = 50
allNum = 100

policy_A = policy.copy()
stateValue_A = stateValue.copy()

for i in range(allNum):
    stateValue_B = Test.policyEvaluation(policy_A,stateValue_A,iterNum)
    #stateValue_B = np.round(stateValue_B,decimals=1)
    #print("stateValue_B：")
    #print(stateValue_B)
    policy_B = Test.policyImproverment(stateValue_B,policy_A)
    #Test.print_policy(policy_B)
    #print("--------------------------------------------------------------------------------")
    policy_A = policy_B.copy()
    stateValue_A = stateValue_B.copy() 

In [97]:
Test.print_policy(policy_B)
print(stateValue_B)

---右 下上左右 --左- 下上左右 --左- 
-上-右 -上-- -上左- -上-- -上左- 
-上-右 -上-- -上左- -上-- -上左- 
-上-右 -上-- -上左- -上-- -上左- 
-上-- -上-- -上-- -上-- -上-- 
[[17.4791 19.4212 17.4791 18.4502 16.6052]
 [15.7312 17.4791 15.7312 16.6052 14.9446]
 [14.1581 15.7312 14.1581 14.9446 13.4502]
 [12.7423 14.1581 12.7423 13.4502 12.1052]
 [10.4681 11.7423 10.4681 11.1052 9.8946]]


**最终得到的策略以及状态价值如上**